In [20]:
import os
from netml.pparser.parser import PCAP
import pprint
from scapy.all import *
from scapy.layers.inet import *
from tqdm import tqdm

import tensorflow as tf


In [21]:
dataDir = "/code/2.Malwares/"
targetFile = "Zeus.pcap"
destination = dataDir + "out"
tsharkPath="/usr/bin/tshark"
packetSizeInBytes = 100
packetsPerFlow = 2
tfrecorsFolder = dataDir + 'tfrecords'
nb_shards = 10
arrOfTfrecordsWriter = [] # un objet criter par shard

In [22]:
cats = ["Benign","Malware"]
listofProcessedFiles = []
training_data = []

In [23]:
for i in range(nb_shards):
    arrOfTfrecordsWriter.append(tf.io.TFRecordWriter(path=tfrecorsFolder + os.sep + 'shard_' + str(i) + '.tfrecords'))

In [24]:
def getLabelForFile(filename):
    filename.replace('.pcap','')
    if filename.find('-') != -1:
        filename = filename.split('-')[0]
    for index,processedFile in enumerate(listofProcessedFiles):
        if processedFile == filename:
            return index
    listofProcessedFiles.append(filename)
    return len(listofProcessedFiles)
    
#pprint.pprint(listofProcessedFiles)
#label = getLabelForFile('tes-test')
#print(label)

In [25]:
def _bytes_feature(value):# peut-être un tableau
    """Returns a bytes_list from a string / byte."""
    if isinstance(value, type(tf.constant(0))):
        value = value.numpy()  # BytesList won't unpack a string from an EagerTensor.
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

In [26]:
def _int64_feature(value):
    """Returns an int64_list from a bool / enum / int / uint."""
    return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))

In [27]:
# Create the features dictionary.
def packet_example(packetArray, label):
    feature = {
        'label': _int64_feature(label),
        'packet1': _bytes_feature(packetArray[0]),
        'packet2': _bytes_feature(packetArray[1]),
    }
    return tf.train.Example(features=tf.train.Features(feature=feature))

In [28]:
 def processPackets(packets):
    newFlow = []
    for i,packet in enumerate(packets):
        packetHex = linehexdump(packet.payload,onlyhex=1,dump=True) # dump packet to hex 01 DE 
        packetHexnoSpace = packetHex.replace(' ','').lower() #to hex 01de
        packetBytes = bytes.fromhex(packetHexnoSpace) # to bytestring
        if len(packetBytes) < packetSizeInBytes: # ajuster les packets à une taille de 100 bytes 
            packetPadded = packetBytes.ljust(packetSizeInBytes,b'\0') #padding
        else:
            packetPadded = packetBytes[:packetSizeInBytes] # couper les paquet trop grand
        if i >= packetsPerFlow:
            break
        newFlow.append(packetPadded)
    return newFlow


In [29]:
tf.data.TFRecordDataset.list_files('2.Malwares/*/*.pcap')

<ShuffleDataset shapes: (), types: tf.string>

In [30]:
import random
for cat in cats: # pour chaque dossier Benign, Malware
    path = os.path.join(dataDir,cat)
    for filePcap in os.listdir(path): #parcours les fichiers pour chaque dossier
        pprint.pprint(path)
        pprint.pprint(filePcap)

        parsedPcap = PCAP(
            os.path.join(path,filePcap),
            flow_ptks_thres=packetsPerFlow,
            verbose=3
        )
        parsedPcap.pcap2flows(interval=1000)
        for flow in tqdm(parsedPcap.flows): #(val,182.168.1.15,,)(<Ether
            flow = flow[1:] # retirer le premier élément car inutile
            for packets in flow:
                writer = random.choice(arrOfTfrecordsWriter)
                tf_example = packet_example(processPackets(packets), getLabelForFile(filePcap))
                writer.write(tf_example.SerializeToString())
                


"""
TODO : convertir hexdump srting to bytestring (https://www.delftstack.com/howto/python/python-convert-hex-to-byte/)
        - ensure only the two first packet are processed ok
        - implement padding ok
        - label each flow 
        - put everything in pickles
        - tweak to treat whole dataset not juste one pcap file

        [[packet 1,
        packet 2],
        label]
"""





'/code/2.Malwares/Benign'
'Outlook.pcap'
'_pcap2flows()' starts at 2021-05-01 13:18:15
pcap_file: /code/2.Malwares/Benign/Outlook.pcap
100%|██████████| 15/15 [00:00<00:00, 2032.52it/s]After splitting flows, the number of subflows: 15 and each of them has at least 2 packets.
'_pcap2flows()' ends at 2021-05-01 13:18:25 and takes 0.1746 mins.
'/code/2.Malwares/Benign'
'Weibo-11.pcap'
'_pcap2flows()' starts at 2021-05-01 13:18:25
pcap_file: /code/2.Malwares/Benign/Weibo-11.pcap

  0%|          | 32/7795 [00:00<00:25, 310.16it/s]After splitting flows, the number of subflows: 7795 and each of them has at least 2 packets.
'_pcap2flows()' ends at 2021-05-01 13:20:16 and takes 1.8375 mins.
100%|██████████| 7795/7795 [00:19<00:00, 395.84it/s]
'/code/2.Malwares/Benign'
'Weibo-3.pcap'
'_pcap2flows()' starts at 2021-05-01 13:20:35
pcap_file: /code/2.Malwares/Benign/Weibo-3.pcap
  1%|          | 44/7901 [00:00<00:18, 430.12it/s]After splitting flows, the number of subflows: 7901 and each of them has

'\nTODO : convertir hexdump srting to bytestring (https://www.delftstack.com/howto/python/python-convert-hex-to-byte/)\n        - ensure only the two first packet are processed ok\n        - implement padding ok\n        - label each flow \n        - put everything in pickles\n        - tweak to treat whole dataset not juste one pcap file\n\n        [[packet 1,\n        packet 2],\n        label]\n'